In [1]:
import json
import os
import shutil
import gzip
import zipfile
import urllib

from rdflib import Graph

urllib.request.urlretrieve(
    "https://data.consilium.europa.eu/data/public-voting/council-votes-on-legislative-acts.zip",
    "data/data.zip")
with zipfile.ZipFile("data/data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")


In [5]:
def process_ttl_gz(path: str):
    print(f"Processing {path}")
    with gzip.open(f"{path}.ttl.gz", 'rb') as f_in:
        with open(f"{path}.ttl", 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    if os.path.exists(path + ".json"):
        file = open(path + ".json")
        data = json.load(file)
        return data
    g = Graph()
    g.parse(path + ".ttl")
    g.serialize(path + ".json", format="json-ld")
    f = open(path + ".json")
    data = json.load(f)
    return data

In [6]:
voting_results = process_ttl_gz("data/VotingResults")
voting_results_cube = process_ttl_gz("data/VotingResultsCube")
voting_results_new = process_ttl_gz("data/VotingResultsNew")

Processing data/VotingResults
Processing data/VotingResultsCube
Processing data/VotingResultsNew


In [10]:
votes = []
num_acts = 0
num_votes_in_favour = 0
num_votes_against = 0
num_votes_abstained = 0
definitions = [x for x in voting_results_cube if "http://www.w3.org/2004/02/skos/core#definition" in x]
for act in definitions:
    act_id = act["@id"]
    votes = [x for x in voting_results_cube if
             "http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act" in x and
             "@id" in x["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act"][0] and
             x["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act"][0][
                 "@id"] == act_id]
    if len(votes) == 0:
        continue
    num_acts += 1
    votes_infavour = [vote for vote in votes if
                      vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                          "@id"].split("/")[-1] == "votedinfavour"]
    num_votes_in_favour += len(votes_infavour)
    votes_against = [vote for vote in votes if
                     vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                         "@id"].split("/")[-1] == "votedagainst"]
    num_votes_against += len(votes_against)
    votes_abstained = [vote for vote in votes if
                       vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                           "@id"].split("/")[-1] == "abstained"]
    num_votes_abstained += len(votes_abstained)

    print(
        f"Found {len(votes)} votes for {act_id} with {len(votes_infavour)} in favour, {len(votes_against)} against and {len(votes_abstained)} abstained ")
    print(act["http://www.w3.org/2004/02/skos/core#definition"][0]["@value"])
    print(
        f"{votes[0]["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/votingrule"][0]["@id"].split("/")[-1]}")


Found 27 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/32481 with 27 in favour, 0 against and 0 abstained 
Council's position on draft amending budget No 1 to the general budget for 2013
qualifiedmajority
Found 28 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/345503 with 26 in favour, 0 against and 0 abstained 
Position of the Council at first reading with a view to the adoption of a Regulation of the European Parliament and of the Council on the European Union Agency for Law Enforcement Cooperation (Europol) and replacing and repealing Council Decisions 2009/371/JHA, 2009/934/JHA, 2009/935/JHA, 2009/936/JHA and 2009/968/JHA
qualifiedmajority
Found 27 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/31224 with 27 in favour, 0 against and 0 abstained 
Directive of the European Parliament and of the Council on indications or marks identifying the lot to which a foodstuff belongs (codified version) (first 

In [11]:
print(f"Total number of acts: {num_acts}")
print(f"Total number of votes in favour: {num_votes_in_favour}")
print(f"Total number of votes against: {num_votes_against}")
print(f"Total number of votes abstained: {num_votes_abstained}")

Total number of acts: 778
Total number of votes in favour: 20806
Total number of votes against: 265
Total number of votes abstained: 278
